In [3]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [4]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [5]:
from functions import *

In [22]:
initial_w = np.zeros(tX.shape[1])
max_iters = 50
gamma = 3e-7

In [23]:
res = least_squares_GD(y, tX, initial_w, max_iters, gamma)

Gradient Descent(0/49): loss=0.5, w0=3.22999485312e-05, w1=-8.185259581200002e-06
Gradient Descent(1/49): loss=0.46849750202717927, w0=4.737735523736878e-05, w1=-8.288091556578184e-06
Gradient Descent(2/49): loss=0.4513712748460005, w0=7.283027314074192e-05, w1=-1.4153561994065593e-05
Gradient Descent(3/49): loss=0.44178360296424807, w0=8.891083213320661e-05, w1=-1.5810090997021515e-05
Gradient Descent(4/49): loss=0.43617479340076165, w0=0.00010996659128578842, w1=-2.0443255627319816e-05
Gradient Descent(5/49): loss=0.4326884948512339, w0=0.0001257725187546786, w1=-2.287872071660321e-05
Gradient Descent(6/49): loss=0.4303539841023342, w0=0.0001437960135669708, w1=-2.6846011158228963e-05
Gradient Descent(7/49): loss=0.4286614226647361, w0=0.0001587507183638464, w1=-2.966032971941446e-05
Gradient Descent(8/49): loss=0.4273411475179239, w0=0.00017453053731563133, w1=-3.325760857123354e-05
Gradient Descent(9/49): loss=0.4262489193466734, w0=0.00018840941370472435, w1=-3.6244983745871636e-0

## Generate predictions and save ouput in csv format for submission:

In [10]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)